In [ ]:
import streamlit as st
import numpy as np
import tensorflow as tf
import os
import pickle
import pandas as pd
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
def build_feature_extractor(model_name):
    if model_name == 'resnet50':
        base_model = ResNet50(weights='imagenet', include_top=False)
    elif model_name == 'vgg16':
        base_model = VGG16(weights='imagenet', include_top=False)
    else:
        raise ValueError("Model not supported.")
    
    base_model.trainable = False
    x = GlobalAveragePooling2D()(base_model.output)
    model = Model(inputs=base_model.input, outputs=x)
    return model

feature_extractor_resnet = build_feature_extractor('resnet50')
feature_extractor_vgg16 = build_feature_extractor('vgg16')

In [12]:
def preprocess_resnet(img):
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = tf.keras.applications.resnet50.preprocess_input(img_array)
    return np.expand_dims(img_array, axis=0)

def preprocess_vgg(img):
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = tf.keras.applications.vgg16.preprocess_input(img_array)
    return np.expand_dims(img_array, axis=0)

In [13]:
signature_database_resnet = {}
signature_database_vgg16 = {}

In [14]:
def store_genuine_signature(user_id, img):
    img_resnet = preprocess_resnet(img)
    features_resnet = feature_extractor_resnet.predict(img_resnet)
    signature_database_resnet[user_id] = features_resnet.reshape(1, -1)
    
    img_vgg = preprocess_vgg(img)
    features_vgg = feature_extractor_vgg16.predict(img_vgg)
    signature_database_vgg16[user_id] = features_vgg.reshape(1, -1)

In [6]:
def verify_signature(user_id, input_img, threshold=0.85):
    if user_id not in signature_database_resnet or user_id not in signature_database_vgg16:
        return "User not found."
    
    stored_resnet = signature_database_resnet[user_id]
    stored_vgg = signature_database_vgg16[user_id]
    
    input_resnet = preprocess_resnet(input_img)
    input_vgg = preprocess_vgg(input_img)
    
    input_features_resnet = feature_extractor_resnet.predict(input_resnet).reshape(1, -1)
    input_features_vgg = feature_extractor_vgg16.predict(input_vgg).reshape(1, -1)
    
    similarity_resnet = cosine_similarity(stored_resnet, input_features_resnet)[0][0]
    similarity_vgg = cosine_similarity(stored_vgg, input_features_vgg)[0][0]
    
    resnet_result = "Genuine" if similarity_resnet >= threshold else "Forged"
    vgg_result = "Genuine" if similarity_vgg >= threshold else "Forged"
    
    return f"ResNet-50: {resnet_result} (Score: {similarity_resnet:.2f})\nVGG-16: {vgg_result} (Score: {similarity_vgg:.2f})"

In [15]:
st.title("Signature Verification System")
user_id = st.text_input("Enter User ID")
uploaded_signature = st.file_uploader("Upload Genuine Signature", type=["png", "jpg", "jpeg"])
uploaded_test_signature = st.file_uploader("Upload Signature to Verify", type=["png", "jpg", "jpeg"])

2025-03-08 08:55:29.608 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 08:55:29.611 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 08:55:29.613 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 08:55:29.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 08:55:29.617 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 08:55:29.618 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 08:55:29.621 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 08:55:29.622 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [8]:
if st.button("Store Genuine Signature"):
    if user_id and uploaded_signature:
        img = image.load_img(uploaded_signature)
        store_genuine_signature(user_id, img)
        st.success(f"Stored genuine signature for user {user_id}.")
    else:
        st.error("Please enter a user ID and upload a signature.")

2025-03-08 08:54:40.876 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 08:54:40.878 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 08:54:40.880 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 08:54:40.881 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 08:54:40.882 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [9]:
if st.button("Verify Signature"):
    if user_id and uploaded_test_signature:
        img = image.load_img(uploaded_test_signature)
        result = verify_signature(user_id, img)
        st.write(result)
    else:
        st.error("Please enter a user ID and upload a signature to verify.")

2025-03-08 08:54:47.042 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 08:54:47.044 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 08:54:47.046 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 08:54:47.047 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-08 08:54:47.050 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
